In [1]:
from collections import defaultdict, Counter
from tqdm import tqdm
import csv
import os
import pandas as pd

In [6]:
ROOT = '/mnt/data0/lucy/manosphere/'
ANN_FILE = ROOT + 'data/ann_sig_entities.csv'
LOGS = ROOT + 'logs/'
COREF = '/mnt/data0/dtadimeti/manosphere/logs/coref_people/'
SUB_META = ROOT + 'data/subreddits.txt'

In [8]:
# Get subreddit categories 
categories = defaultdict(str)
categories_rev = defaultdict(list)
with open(SUB_META, 'r') as infile: 
    reader = csv.DictReader(infile)
    for row in reader: 
        name = row['Subreddit'].strip().lower()
        if name.startswith('/r/'): name = name[3:]
        if name.startswith('r/'): name = name[2:]
        if name.endswith('/'): name = name[:-1]
        categories[name] = row['Category after majority agreement']
        categories_rev[row['Category after majority agreement']].append(name)

In [25]:
df = pd.read_csv(LOGS + 'pronoun_df.csv')
df['category'] = df['community'].map(categories) 
df = df[~df.category.isin(['Health', 'Criticism'])]
df

,month,community,word,fem,masc,neut,category
0,2011-07,seduction,girl,762,6,1,PUA
1,2011-07,mensrights,young boy,0,2,0,MRA
2,2011-07,intactivists,son,0,2,0,MRA
3,2011-07,mensrights,man,6,491,0,MRA
4,2011-07,mensrights,girlfriend,10,3,0,MRA
...,...,...,...,...,...,...,...
590300,2010-04,seduction,poor girl,1,0,0,PUA
590301,2010-04,seduction,random guy,0,1,0,PUA
590302,2010-04,seduction,other girl,1,0,0,PUA
590303,2010-04,seduction,good girl,1,0,0,PUA


In [43]:
def show_top_fem(cat): 
    cat_df = df[df.category == cat]
    cat_totals = cat_df.groupby('word').sum()
    cat_totals['total'] = cat_totals['fem'] + cat_totals['masc'] 
    # filter to only those that appear at least 10 times as she or he
    cat_totals = cat_totals[cat_totals['total'] > 10] 
    cat_totals['fem_frac'] = cat_totals['fem'] / (cat_totals['fem'] + cat_totals['masc'])
    cat_fem = cat_totals[cat_totals['fem_frac'] == 1]
    return cat_fem.sort_values(by=['total'])

In [44]:
# The Red Pill
show_top_fem('TRP')

,fem,masc,neut,total,fem_frac
word,,,,,
popular girl,11,0,0,11,1.0
old bitch,11,0,0,11,1.0
naked woman,11,0,0,11,1.0
nagging wife,11,0,0,11,1.0
midwife,11,0,0,11,1.0
...,...,...,...,...,...
first wife,201,0,0,201,1.0
beautiful woman,221,0,1,221,1.0
attractive woman,264,0,4,264,1.0


In [45]:
# Men's Rights Activists
show_top_fem('MRA')

,fem,masc,neut,total,fem_frac
word,,,,,
normal woman,11,0,0,11,1.0
straight woman,11,0,0,11,1.0
wonderful woman,11,0,0,11,1.0
older lady,11,0,1,11,1.0
cleaning lady,11,0,0,11,1.0
...,...,...,...,...,...
same woman,100,0,0,100,1.0
teenage girl,106,0,0,106,1.0
married woman,111,0,0,111,1.0
